# Проект по SQL

**Цель:** анализ базы данных

Необходимо:
* Вычислить, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определите издательство, которое выпустило наибольшее число книг толще 50 страниц (исключив из анализа брошюры);
* Определить автора с самой высокой средней оценкой книг (с учетом книг с 50 и более оценками);
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Описание данных:
* Таблица books: 
  * book_id — идентификатор книги; 
  * author_id — идентификатор автора;
  * title — название книги; 
  * num_pages — количество страниц; 
  * publication_date — дата публикации книги; 
  * publisher_id — идентификатор издателя.
* Таблица authors: 
  * author_id — идентификатор автора; 
  * author — имя автора.
* Таблица publishers: 
  * publisher_id — идентификатор издательства; 
  * publisher — название издательства;
* Таблица ratings: 
  * rating_id — идентификатор оценки; 
  * book_id — идентификатор книги; 
  * username — имя пользователя, оставившего оценку; 
  * rating — оценка книги.
* Таблица reviews: 
  * review_id — идентификатор обзора; 
  * book_id — идентификатор книги; 
  * username — имя автора обзора; 
  * text — текст обзора.

## Загрузка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Просмотр общей информации о таблицах

In [2]:
display(pd.io.sql.read_sql("SELECT * FROM books LIMIT 1", con = engine))
display(pd.io.sql.read_sql("SELECT * FROM authors LIMIT 1", con = engine))
display(pd.io.sql.read_sql("SELECT * FROM publishers LIMIT 1", con = engine))
display(pd.io.sql.read_sql("SELECT * FROM ratings LIMIT 1", con = engine))
display(pd.io.sql.read_sql("SELECT * FROM reviews LIMIT 1", con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


,author_id,author
0,1,A.S. Byatt


,publisher_id,publisher
0,1,Ace


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


##  Количество книг, вышедших после 1 января 2000 года

In [3]:
pd.io.sql.read_sql('''SELECT COUNT(*) AS cnt 
                      FROM books 
                      WHERE CAST(publication_date AS date) > '2000-01-01'; ''', con = engine)

,cnt
0,819


Всего было выпущено 819 книг после 1 января 2000 года. 

## Количество обзоров и средняя оценка каждый книги

In [3]:
pd.io.sql.read_sql('''SELECT books.title AS title, 
                             COUNT(DISTINCT(reviews.review_id)) AS cnt_reviews, 
                             AVG(ratings.rating) AS avg_rating 
                      FROM reviews 
                      LEFT JOIN ratings ON reviews.book_id = ratings.book_id
                      LEFT JOIN books ON reviews.book_id = books.book_id
                      GROUP BY books.title;''', con = engine)

,title,cnt_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
988,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
989,Xenocide (Ender's Saga #3),3,3.400000
990,Year of Wonders,4,3.200000
991,You Suck (A Love Story #2),2,4.500000


Получили информацию о том, сколько обзоров было сделано на каждую книгу, и среднюю оценку каждой. 

## Издательство, которое выпустило наиболее количество книг, толще 50 страниц

In [5]:
pd.io.sql.read_sql('''SELECT publishers.publisher AS publisher, 
                             COUNT(books.book_id) AS cnt_books 
                      FROM publishers 
                      LEFT JOIN books on publishers.publisher_id = books.publisher_id 
                      WHERE books.num_pages > 50 
                      GROUP BY publishers.publisher
                      ORDER BY cnt_books DESC; ''', con = engine)

,publisher,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Издательство Penguin Books выпустило наибольшее количество книг, толщиной более 50 страниц. Они выпустили 42 книги. 

## Автор с самой высокой средней оценкой книг (с учетом книг с 50 и более оценками)

In [46]:
pd.io.sql.read_sql('''SELECT AVG(ratings.rating) AS avg_rating,
                             authors.author AS author
                      FROM(
                           SELECT book_id AS book_id,
                                  COUNT(rating_id) AS cnt_rating
                           FROM ratings
                           GROUP BY book_id
                           HAVING COUNT(rating_id) >= 50) AS sub
                      LEFT JOIN ratings ON ratings.book_id = sub.book_id
                      LEFT JOIN books ON books.book_id = sub.book_id
                      LEFT JOIN authors ON authors.author_id = books.author_id
                      GROUP BY authors.author
                      ORDER BY avg_rating DESC; ''', con = engine)


,avg_rating,author
0,4.287097,J.K. Rowling/Mary GrandPré
1,4.264151,Markus Zusak/Cao Xuân Việt Khương
2,4.246914,J.R.R. Tolkien
3,4.192308,Louisa May Alcott
4,4.080645,Rick Riordan
5,3.901408,William Golding
6,3.825581,J.D. Salinger
7,3.789474,Paulo Coelho/Alan R. Clarke/Özdemir İnce
8,3.787879,William Shakespeare/Paul Werstine/Barbara A. M...
9,3.750000,Lois Lowry


J.K. Rowling/Mary GrandPré - автор с наибольшей средней оценкой (4.3), с учетом книг, у которых 50 и более оценок. 

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [59]:
pd.io.sql.read_sql('''SELECT AVG(cnt_reviews) AS avg_reviews
                      FROM (
                            SELECT COUNT(reviews.review_id) AS cnt_reviews,
                                   username AS username
                            FROM reviews
                            WHERE reviews.username IN (
                                                       SELECT ratings.username
                                                       FROM ratings
                                                       GROUP BY ratings.username
                                                       HAVING COUNT(rating_id) >= 50)
                            GROUP BY username) AS sub
                      ''', con = engine)


,avg_reviews
0,24.222222


В среднем, пользователи, которые поставили больше 50 оценок, оставляют 24 обзора на книги. 

## Общие выводы

Было посчитано:

* Количество книг, вышедших после 1 января 2000 года, - 819 штук;
* Для каждой книги было посчитано количество сделанных обзоров и средняя оценка;
* Было определено издательство, которое выпустило наибольшее число книг толще 50 страниц, - "Penguin Books";
* Автор с самой высокой средней оценкой книг (с учетом книг с 50 и более оценками) - J.K. Rowling/Mary GrandPré;
* Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, - 24 штуки. .